In [10]:
import sys
import pandas as pd 
import numpy as np

from itertools import chain
from pymongo import MongoClient


In [3]:
# to make interactive plotting possible
%matplotlib inline
# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

path_source = "../"
sys.path.append(path_source)

In [4]:
from utils.misc import read_mongo, batch
from utils.etl import get_all_items_chunk
from utils.dk021 import DK021

In [5]:
batch_size = 50_000

# connect to tenders db
cursor = MongoClient()['prozorro']['etenders'].find(no_cursor_timeout=True)

c = None

try:
    for idx, b in enumerate(batch(cursor, batch_size)):

        all_items_chunk = get_all_items_chunk(
            df=pd.DataFrame(b).rename(columns={'_id':'id'})
        )
        if all_items is None:
            all_items = all_items_chunk
        else:
            all_items = pd.concat([all_items, all_items_chunk])

        print(f'chunk {idx}')
finally:
    cursor.close()

chunk 0
chunk 1
chunk 2
chunk 3
chunk 4
chunk 5
chunk 6
chunk 7
chunk 8
chunk 9
chunk 10
chunk 11
chunk 12
chunk 13
chunk 14
chunk 15
chunk 16
chunk 17
chunk 18
chunk 19
chunk 20
chunk 21
chunk 22
chunk 23
chunk 24
chunk 25
chunk 26


In [6]:
all_items.to_parquet(f'../data/search/all_items.parquet.gzip', compression='gzip') 

In [7]:
all_items.shape

(1479900, 4)

In [8]:
full_df = pd.read_parquet(f'../data/enreached/full.parquet.gzip')

In [11]:
get_first = lambda x: x.iloc[0] if x.iloc[0] is not None else []
flatten_unique = lambda x: list(set(chain.from_iterable(list(x))))

suppliers_df = (
    full_df
    .query('bids_tenderers_identifier_scheme=="UA-EDR"')
    .merge(
        all_items,
        left_on=['id', 'lots_id'], 
        right_on=['all_items_parent_id', 'all_items_relatedLot'],
        how='inner'
        )
    [[
        "bids_tenderers_name",
        "bids_tenderers_identifier_id",
        "bids_tenderers_contactPoint_email",
        "bids_tenderers_contactPoint_name",
        "bids_tenderers_contactPoint_telephone",
        "bids_tenderers_geo",
        "all_items_code",
        "all_items_description",
        "id"
    ]]
    .assign(
        lat = lambda d: d.bids_tenderers_geo.apply(lambda geo:geo[0] if geo is not None else np.inf),
        lon = lambda d: d.bids_tenderers_geo.apply(lambda geo:geo[1] if geo is not None else np.inf),
    )
    .groupby(by=['bids_tenderers_identifier_id'])
    .agg(
        company_name = ('bids_tenderers_name', get_first), 
        contact_email = ('bids_tenderers_contactPoint_email', get_first),
        contact_name = ('bids_tenderers_contactPoint_name', get_first),
        contact_telephone = ('bids_tenderers_contactPoint_telephone', get_first),
        lat = ('lat', get_first),
        lon = ('lon', get_first),
        product_code= ("all_items_code", flatten_unique),
        product_description = ('all_items_description', flatten_unique)
    )
    .assign(
        product_code = lambda d: d.product_code.apply(lambda codes:[DK021.split_code(code=code, sep='/') for code in codes]),
        # TODO needs cleaning, deduplication
        product_description = lambda d: d.product_description.apply(lambda descriptions:" ".join(descriptions)),
    )
    .reset_index()
    .rename(columns={
        "bids_tenderers_identifier_id": "edr_code"
    })
)

In [13]:
suppliers_df.head(10)

,edr_code,company_name,contact_email,contact_name,contact_telephone,lat,lon,product_code,product_description
0,,ФІЗИЧНА ОСОБА-ПІДПРИЄМЕЦЬ САВЧУК-ЦАРИНСЬКА ОКС...,oksanasavchuknew@gmail.com,ФІЗИЧНА ОСОБА-ПІДПРИЄМЕЦЬ САВЧУК-ЦАРИНСЬКА ОКС...,+380986695030,50.479131,24.277054,[79/5/3/0/000],Послуги з письмового перекладу
1,31681667,"ПП ""ІДЕАЛ""",e.mischenko@fortuna.odessa.ua,Мiщенко Олена Миколаївна,+380675593194,46.487319,30.739278,"[15/6/1/0/000, 15/8/2/0/000, 15/3/3/0/000, 15/...","""Продукція борошномельно-круп'яної промисловос..."
2,0000231271,UNIBEP S.A.,olptolpt@gmail.com,Олег Петровский,+380503222407,49.070498,27.673713,"[45/0/0/0/000, 45/2/2/0/000]",Реконструкція міжнародного пункту пропуску для...
3,2105615855,ФОП Бихун Віктор Юрійович,lk@ukr.net,Бихун Віктор Юрійович,+380503102204,50.587515,30.489363,[22/1/1/0/000],Книга пам’яті військовослужбовців Збройних сил...
4,2133907313,ФОП Шевцов Ю.Г.,shefug4@gmail.com,Шевцов Юрий Григорьевич,+380676333482,48.635851,35.259554,"[72/7/1/0/000, 50/3/1/3/000, 30/2/1/0/000]",Системний блок Налаштування внутрішньої комп`...
5,31437305,"ООО ""Компания производственное предприятие ""СВАН""",sales@svan.com.ua,"ООО ""Компания производственное предприятие ""СВАН""",+380577509077,inf,inf,[18/1/0/0/000],Футболки та сорочки Літні комбінезони кепки
6,32447450,"ТОВ ""Факультет""",apteka.dostavka@gmail.com,Тимків Марія,+380509329770,48.468022,35.041771,[33/6/0/0/000],"ДИП РИЛІФ, гель по 100 гр. у тубі Бетаргин р-..."
7,34477398,"ПП ""КИЇВ-ПІВДЕНЬ""",pavel-step@ukr.net,Плетенков Павел,+380963226699,44.601051,33.522914,[03/1/2/0/000],Квіти для забезпечення організації та проведен...
8,3602309565,ФОП Байдак Ж.В.,baidak.vasil@gmail.com,Байдак Василь Миколайович,+380675416071,47.910273,33.391770,[71/2/5/0/000],Капітальний ремонт системи господарсько-побуто...
9,38767462,"ТЗОВ ""АРІЄС - УКРАЇНА""",teply_e@arjes.com.ua,Теплий Євген Ігорович,+380673404830,49.960262,24.814713,[16/6/0/0/000],Побрібнювач деревини


In [14]:
suppliers_df.to_parquet(f'../data/search/suppliers.parquet.gzip', compression='gzip') 

In [75]:
suppliers_df = pd.read_parquet(f'../data/search/suppliers.parquet.gzip')
suppliers_df.shape